# Python for (open) Neuroscience

_Lecture 1.4_ - Advanced `pandas`

Luigi Petrucco

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec-2025/blob/main/lectures/Lecture1.4_More-pandas.ipynb)

## Outline
 - organise data in dataframes
 - aggregate statistics with `groupby()`
 - index broadcasting in `pandas`

## Organize data in `DataFrame`s

A very common issue in data science is: how to organize our datasets?

In [1]:
import numpy as np
import pandas as pd 

n_subjects = 4
n_repetitions = 3
subject_data = [{"accuracy":np.random.rand(), "speed":np.random.rand()}  for _ in range(n_repetitions)]

In [2]:
# Imagine we present subject with three trials of a stimulus. 
# during every trial, we measure 2 variables (eg, accuracy and speed).

# We could represent the data entries for each stimulus as a dictionary, 
# and the data for all trials for a subject as a list of dictionaries:
subject_data

[{'accuracy': 0.09996035101463974, 'speed': 0.8585137490231265},
 {'accuracy': 0.48326152353436724, 'speed': 0.24702362216237506},
 {'accuracy': 0.1210487235239599, 'speed': 0.18461502163756083}]

In [4]:
all_subjects_data = dict()

for i in range(n_subjects):
    all_subjects_data[f"subj_{i}"] = \
        [{"accuracy":np.random.rand(), "speed":np.random.rand()} for _ in range(n_repetitions)]
all_subjects_data

{'subj_0': [{'accuracy': 0.057807243218939375, 'speed': 0.38368992153061565},
  {'accuracy': 0.7018136203916324, 'speed': 0.5219865762537793},
  {'accuracy': 0.42324592753334833, 'speed': 0.3995187435442098}],
 'subj_1': [{'accuracy': 0.4588406948079047, 'speed': 0.4719216533691065},
  {'accuracy': 0.4521411072256021, 'speed': 0.6462947820108449},
  {'accuracy': 0.24268994910894104, 'speed': 0.8630395004087745}],
 'subj_2': [{'accuracy': 0.3213749816439092, 'speed': 0.7894085477631394},
  {'accuracy': 0.19715228668510676, 'speed': 0.980833861225406},
  {'accuracy': 0.5432855203766879, 'speed': 0.6362355569642426}],
 'subj_3': [{'accuracy': 0.7231242547576374, 'speed': 0.3947917305717832},
  {'accuracy': 0.48534070891402115, 'speed': 0.33824092672948614},
  {'accuracy': 0.8723151655435882, 'speed': 0.6625089028536947}]}

In [5]:
# Imagine we have 4 experimental subjects: we could pool the data 
# for all subjects as a dictionary of lists of dictionaries.

from pprint import pp  # Pretty print function! Just learned about this
pp(all_subjects_data)

{'subj_0': [{'accuracy': 0.057807243218939375, 'speed': 0.38368992153061565},
            {'accuracy': 0.7018136203916324, 'speed': 0.5219865762537793},
            {'accuracy': 0.42324592753334833, 'speed': 0.3995187435442098}],
 'subj_1': [{'accuracy': 0.4588406948079047, 'speed': 0.4719216533691065},
            {'accuracy': 0.4521411072256021, 'speed': 0.6462947820108449},
            {'accuracy': 0.24268994910894104, 'speed': 0.8630395004087745}],
 'subj_2': [{'accuracy': 0.3213749816439092, 'speed': 0.7894085477631394},
            {'accuracy': 0.19715228668510676, 'speed': 0.980833861225406},
            {'accuracy': 0.5432855203766879, 'speed': 0.6362355569642426}],
 'subj_3': [{'accuracy': 0.7231242547576374, 'speed': 0.3947917305717832},
            {'accuracy': 0.48534070891402115, 'speed': 0.33824092672948614},
            {'accuracy': 0.8723151655435882, 'speed': 0.6625089028536947}]}


This is now organized but very **nested**! it is not easy to perform statistics on it.

In [6]:
# Imagine we want to average the results across all subjects for variable_1:
means = []
for subject_results in all_subjects_data.values():
    for result in subject_results:
        means.append(result["speed"])
np.mean(means)

0.5907058919354236

When we organize data in pandas dataframes, there is an important principle to keep in mind:

**keep them as flat as possible**

`flat` = opposite of nested

`nested` = lists of dictionaries of lists of dictionaries of dataframes of...



Remember!


    🪷 The Zen of Python 🪷
        
        Flat is better than nested

In [7]:
# We can turn the data into a dataframe (does not matter how we do it here! this is just an ugly example)
flat_list_of_dicts = []

for sub in all_subjects_data.keys():
    for n_rep in range(n_repetitions):
        trial_dict = all_subjects_data[sub][n_rep]
        trial_dict.update({"subject": sub, "repetition": n_rep})
        
        flat_list_of_dicts.append(trial_dict)
                    
trials_df = pd.DataFrame(flat_list_of_dicts)
trials_df


,accuracy,speed,subject,repetition
0,0.057807,0.383690,subj_0,0
1,0.701814,0.521987,subj_0,1
2,0.423246,0.399519,subj_0,2
3,0.458841,0.471922,subj_1,0
4,0.452141,0.646295,subj_1,1
5,0.242690,0.863040,subj_1,2
6,0.321375,0.789409,subj_2,0
7,0.197152,0.980834,subj_2,1
8,0.543286,0.636236,subj_2,2
9,0.723124,0.394792,subj_3,0


We can now easily perform statistics on the data:

In [8]:
trials_df["speed"].mean()

0.5907058919354236

Or look for _e.g._ max and min values across the dataset:

In [11]:
max_idx = trials_df["speed"].argmax()
trials_df.loc[max_idx, :]  # shows subject and trial with max speed across the dataset

accuracy      0.197152
speed         0.980834
subject         subj_2
repetition           1
Name: 7, dtype: object

### Principles for organizing `pandas` dataframes

Keep in the same dataset **all the data of the same type** you have **across groups**.

E.g.:
 - A `subject_df` DataFrame for all subjects, across experimental groups
 - A `trial_df` DataFrame for all trials across subjects
 - A `response_df` DataFrame with responses across areas and across subjects

If you load lists of dataframes (e.g., you have separate files for each subject) concatenate them before start working with them!!

## Split different information over dataframes

Consider having multiple dataframes to describe different aspects of your experiment. For example:

- a `subject_dataframe` with the info on your subjects
- a `trials_dataframe` with the trial responses across subjects

And keep consistent ids / nomenclature to work across both!

Example:

In [12]:
import numpy as np
import pandas as pd
np.random.seed(0)
subjects_df = pd.DataFrame({"sex":np.random.choice(["F", "M"], size=n_subjects),
                            "handedness": np.random.choice(["left", "right"], size=n_subjects),
                            "age": np.random.randint(20, 40, size=n_subjects)})
subjects_df.index = [f"subj_{i}" for i in range(n_subjects)]

In [13]:
# This is a subjects dataframe for the experiment above:
subjects_df

,sex,handedness,age
subj_0,F,right,29
subj_1,M,right,39
subj_2,M,right,38
subj_3,F,right,24


We can now easily filter the subjects we want to work on with smart indexing:

In [14]:
selected_subjects_df = subjects_df[(subjects_df["age"] <= 30)]

print(selected_subjects_df)

       sex handedness  age
subj_0   F      right   29
subj_3   F      right   24


In [15]:
# Take all the selected ids:
sel_subjects_ids = selected_subjects_df.index
sel_subjects_ids

Index(['subj_0', 'subj_3'], dtype='object')

Now we can restrain our analysis on the trials of these subjects :

In [16]:
trials_df.head()

,accuracy,speed,subject,repetition
0,0.057807,0.383690,subj_0,0
1,0.701814,0.521987,subj_0,1
2,0.423246,0.399519,subj_0,2
3,0.458841,0.471922,subj_1,0
4,0.452141,0.646295,subj_1,1


In [17]:
# Here, we'll use another handy pandas method: `.isin()`:

selection = trials_df["subject"].isin(sel_subjects_ids)
selection


0      True
1      True
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9      True
10     True
11     True
Name: subject, dtype: bool

In [18]:
trials_df.loc[selection, "speed"].mean()  # mean speed for subjects < 30 y.o.

0.4501228002472615

## When not to use `pandas`

You do not always need pandas dataframes! E.g., it is not efficient with many columns.

Many times large raw data (ephys, imaging, videos...) can live in numpy arrays and you work in `pandas` only with  derived quantities (after eving extracted responses, metrics, etc.).

(Practicals 1.4.0)

### `.groupby()`

Using flat datasets allows us do operations within categories with `.groupby()`:

The sintax is :
```python
df.groupby("name_of_the_category_column").operation()
```

Let's go back to the dataframe with trieal data of four subjects:

In [19]:
trials_df 

,accuracy,speed,subject,repetition
0,0.057807,0.383690,subj_0,0
1,0.701814,0.521987,subj_0,1
2,0.423246,0.399519,subj_0,2
3,0.458841,0.471922,subj_1,0
4,0.452141,0.646295,subj_1,1
5,0.242690,0.863040,subj_1,2
6,0.321375,0.789409,subj_2,0
7,0.197152,0.980834,subj_2,1
8,0.543286,0.636236,subj_2,2
9,0.723124,0.394792,subj_3,0


Imagine that we want the average over subjects. `subject` will be our grouping column, and the operation is `mean()`:

In [20]:
subj_means_df = trials_df.groupby("subject").mean()
subj_means_df

,accuracy,speed,repetition
subject,,,
subj_0,0.394289,0.435065,1.0
subj_1,0.384557,0.660419,1.0
subj_2,0.353938,0.802159,1.0
subj_3,0.693593,0.465181,1.0


Note how the result will have **the `groupby` variable as index**

If we want the average across subjects for every experimental trial, we write:

In [22]:
repetition_means_df = trials_df.drop(["subject"], axis=1).groupby("repetition").mean()
repetition_means_df

,accuracy,speed
repetition,,
0,0.390287,0.509953
1,0.459112,0.621839
2,0.520384,0.640326


## Index broadcasting in `pandas`

We can do operations across pandas dataframes. When we do so, `pandas` uses the column names and labels to match the elements:

#### Matching over columns

In [23]:
df_a = pd.DataFrame(dict(col_a=[10, 20], col_b=[15, 25]))
df_b = pd.DataFrame(dict(col_b=[1, 2], col_a=[3, 4]))

In [24]:
df_a

,col_a,col_b
0,10,15
1,20,25


In [25]:
df_b

,col_b,col_a
0,1,3
1,2,4


In [26]:
# The result of this operation matched columns based on their labels, not their order!
df_a - df_b

,col_a,col_b
0,7,14
1,16,23


#### Matching over rows

In [27]:
df_a = pd.DataFrame(dict(col_a=[10, 20], col_b=[15, 25]), index=["trial0", "trial1"])
df_b = pd.DataFrame(dict(col_b=[1, 2], col_a=[3, 4]), index=["trial1", "trial0"])

In [28]:
df_a

,col_a,col_b
trial0,10,15
trial1,20,25


In [29]:
df_b

,col_b,col_a
trial1,1,3
trial0,2,4


In [30]:
# The result of this operation matched columns based on their labels, 
# and rows based on their index!
df_a - df_b

,col_a,col_b
trial0,6,13
trial1,17,24


#### Mismatching indexes/columns

When `pandas` does not find matching indexes and columns, it leaves `nan`:

In [31]:
df_a = pd.DataFrame(dict(col_a=[10, 20], col_b=[15, 25]), index=["trial0", "trial1"])
df_b = pd.DataFrame(dict(col_b=[1, 2], col_a=[3, 4], col_c=[0, 0]), index=["trial1", "trial2"])

In [32]:
df_a

,col_a,col_b
trial0,10,15
trial1,20,25


In [33]:
df_b

,col_b,col_a,col_c
trial1,1,3,0
trial2,2,4,0


In [34]:
df_a - df_b

,col_a,col_b,col_c
trial0,NaN,NaN,NaN
trial1,17.0,24.0,NaN
trial2,NaN,NaN,NaN


## Combine `groupby` and broadcasting

Let's normalize the dataframe by subtracting from each subject the means across trials!

In [35]:
# The groupby operation left us with a nice df with subjects as index:
subj_means_df = trials_df.groupby("subject").mean()
subj_means_df.head(3)

,accuracy,speed,repetition
subject,,,
subj_0,0.394289,0.435065,1.0
subj_1,0.384557,0.660419,1.0
subj_2,0.353938,0.802159,1.0


In [36]:
# But our trials dataframe has a numerical index:
trials_df.head(5)

,accuracy,speed,subject,repetition
0,0.057807,0.383690,subj_0,0
1,0.701814,0.521987,subj_0,1
2,0.423246,0.399519,subj_0,2
3,0.458841,0.471922,subj_1,0
4,0.452141,0.646295,subj_1,1


In [37]:
# this will try to match indexes of one dataframe with the other. As there won't be matches, 
# the result will be lots of nans!
trials_df - subj_means_df

,accuracy,repetition,speed,subject
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [38]:
# But we reindex the trials dataframe to make indexes consistent!
trials_df.set_index("subject")

,accuracy,speed,repetition
subject,,,
subj_0,0.057807,0.383690,0
subj_0,0.701814,0.521987,1
subj_0,0.423246,0.399519,2
subj_1,0.458841,0.471922,0
subj_1,0.452141,0.646295,1
subj_1,0.242690,0.863040,2
subj_2,0.321375,0.789409,0
subj_2,0.197152,0.980834,1
subj_2,0.543286,0.636236,2


So now we can write:

In [41]:
normalized_trials_df = trials_df.set_index("subject") - trials_df.groupby("subject").mean()
normalized_trials_df.reset_index()

,subject,accuracy,speed,repetition
0,subj_0,-0.336482,-0.051375,-1.0
1,subj_0,0.307525,0.086921,0.0
2,subj_0,0.028957,-0.035546,1.0
3,subj_1,0.074283,-0.188497,-1.0
4,subj_1,0.067584,-0.014124,0.0
5,subj_1,-0.141867,0.202621,1.0
6,subj_2,-0.032563,-0.012751,-1.0
7,subj_2,-0.156785,0.178675,0.0
8,subj_2,0.189348,-0.165924,1.0
9,subj_3,0.029531,-0.070389,-1.0


From each entry in the trials dataframe, numpy will broadcast the correct value of the mean for that subject!

(Practicals 1.4.1)

## Multi-indexing

Sometimes, we might want to average keeping the segregation over multiple categories:

In [ ]:
# Create again a trials_df:
trials_df = pd.DataFrame(dict(subject=[f"subj{i}" for i in range(n_subjects) for _ in range(n_trials)],
                              trial_type=[f"type{n//2}" for i in range(n_subjects) for n in range(n_trials)],
                              accuracy=np.random.uniform(0, 1, n_trials*n_subjects),
                              rt=np.random.uniform(0, 100, n_trials*n_subjects)))

trials_df.head()

In [ ]:
trial_subj_avg = trials_df.groupby(["subject", "trial_type"]).mean()
trial_subj_avg

In [ ]:
trials_df.set_index(["subject", "trial_type"]) - trial_subj_avg

## Rolling functions with `.rolling()`

Imagine we have a time series of data, and we want to compute the mean in a sliding window (e.g., for smoothing).

In [ ]:
# Let's create a time series:
time_series = pd.Series(np.random.rand(100))
time_series.plot()

In [ ]:
# This will compute the mean in a rolling window - ie, smooth it!
rolling_wnd_size = 10
smoothed = time_series.rolling(rolling_wnd_size).mean()

In [ ]:
time_series.plot(legend="Original")
smoothed.plot(legend="Time averaged")

Note that there will be nan values at the borders, where we do not have enough data to compute the mean

By default, the window will not be centered:

In [ ]:
dirac_series = pd.Series(np.zeros(30))
dirac_series[15] = 1

In [ ]:
dirac_series.plot(figsize=(3,2))
dirac_series.rolling(8).mean().plot()

We can center the window with the `center=True` argument:

In [ ]:
dirac_series.plot(figsize=(3,2))
dirac_series.rolling(8, center=True).mean().plot()

### Usage of `.rolling()`

When done with averaging, same results as other smoothing tools

But now we can use arbitrary functions! (standard deviation, significance tests, etc)

For example, we can look at the range of an oscillation with rolling `max()` and `min()`:

In [ ]:
win_size = 10
time_series.plot()
time_series.rolling(window=win_size, center=True).min().plot()
time_series.rolling(window=win_size, center=True).max().plot()

(Practicals 1.4.2)